# 第6章: 予知（段階的思考）

- [レッスン](#lesson)
- [演習](#exercises)
- [例題プレイグラウンド](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

誰かがあなたを起こして、すぐにいくつかの複雑な質問をし、それにすぐに答えなければならないとしたら、どのようにするでしょうか？おそらく、**最初に答えをよく考える時間が与えられた場合**ほどうまくいかないでしょう。

実は、Claude も同じなのです。

**Claude に段階的に考える時間を与えることで、Claude がより正確になることがあります**。特に複雑なタスクにおいてそうです。しかし、**思考は声に出した時のみカウントされます**。Claude に考えるように求めるが答えのみを出力させることはできません。この場合、実際には思考は発生していません。

### 例

以下のプロンプトでは、人間の読者には2番目の文が最初の文を裏切っていることが明らかです。しかし、**Claude は「unrelated（無関係）」という言葉をあまりに文字通りに受け取ってしまいます**。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

Claude のレスポンスを改善するために、**Claude に答える前にまず物事をよく考えさせましょう**。これは、Claude がタスクを処理し考え抜くために踏むべきステップを文字通り説明することで行います。役割プロンプティングと組み合わせることで、Claude がレビューをより深く理解できるようになります。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude は時々順序に敏感です**。この例はClaude の微妙なテキストを理解する能力の境界にあり、前の例から引数の順序を入れ替えて負を最初に、正を2番目にすると、Claude の全体的な評価が正に変わります。

ほとんどの状況で（しかし混乱することに全てではありませんが）、**Claude は2つの選択肢のうち2番目を選ぶ可能性が高く**、これはWebからの訓練データにおいて2番目の選択肢が正しい可能性が高かったためと考えられます。

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**Claude に考えさせることで、Claude の答えを間違いから正解に変えることができます**。Claude が間違いを犯す多くの場合、それはこれほど単純なことなのです！

Claude の答えが間違っている例を見て、Claude に考えるように求めることでそれをどう修正できるかを確認しましょう。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Claude に段階的に考えるように求めて、今度は`<brainstorm>`タグを使ってこれを修正しましょう。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

上記の内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**例題プレイグラウンド**](#example-playground)をご覧ください。

---

## 演習
- [演習 6.1 - メール分類](#exercise-61---classifying-emails)
- [演習 6.2 - メール分類の書式設定](#exercise-62---email-classification-formatting)

### 演習 6.1 - メール分類
この演習では、Claude にメールを以下のカテゴリに分類するよう指示します：										
- (A) 販売前質問
- (B) 破損または欠陥品
- (C) 請求に関する質問
- (D) その他（説明してください）

演習の最初の部分では、`PROMPT`を変更して、**Claude が正しい分類のみを出力するようにしてください**。あなたの答えには、**正しい選択の文字（A - D）を括弧付きで、そしてカテゴリ名を含める必要があります**。

各メールがどのカテゴリに分類されるべきかを知るために、`EMAILS`リストの各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
print(hints.exercise_6_1_hint)

まだ詰まっていますか？下のセルを実行して例の解決策を見てください。

In [ ]:
print(hints.exercise_6_1_solution)

### 演習 6.2 - メール分類の書式設定
この演習では、上記のプロンプトの出力を改良して、私たちが望む通りに正確に書式設定された答えを得ます。

お気に入りの出力書式設定テクニックを使用して、Claude に正しい分類の文字のみを`<answer></answer>`タグで囲ませてください。例えば、最初のメールの答えは正確に`<answer>B</answer>`という文字列を含む必要があります。

各メールに対してどの文字カテゴリが正しいかを忘れた場合は、`EMAILS`リストの各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、下のセルを実行してください！

In [ ]:
print(hints.exercise_6_2_hint)

### おめでとうございます！

ここまでのすべての演習を解けた場合、次の章に進む準備ができています。楽しいプロンプト作成を！

---

## 例題プレイグラウンド

これは、このレッスンで紹介されたプロンプト例を自由に試し、プロンプトを調整してClaude のレスポンスにどのような影響を与えるかを確認するエリアです。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))